In [1]:
import sys
sys.path.append("/home/tagore/repos/ai/scripts")
import os
import time
import ollama
import chromadb
import torch
import h5py
from transformers import AutoTokenizer, AutoModel
import generate_embeddings as ge

In [2]:
documents_directory = '/home/tagore/repos/ai/data/processed/processed_texts/Bookshelf_NBK279690.txt'
collection_name = "blast_db"
database_file = '/home/tagore/repos/ai/data/processed/embeddings/chrome_db'
documents_directory2 = '/home/tagore/repos/ai/data/processed/texts/Bookshelf_NBK279690.pdf'
collection_name2 = "blast_db_unprocessed"
database_file2 = '/home/tagore/repos/ai/data/processed/embeddings/chrome_db_unprocessed'
collection_name3 = "BioBERT"
database_path3 = '/home/tagore/repos/ai/data/processed/embeddings/chroma_db_withBioBERT'

In [4]:
client = chromadb.PersistentClient(path=database_file)
collection = client.get_collection(name=collection_name)
client2 = chromadb.PersistentClient(path=database_file2)
collection2 = client2.get_collection(name=collection_name2)
client33 = chromadb.PersistentClient('/home/tagore/repos/ai/data/processed/embeddings/BioBERT')
collection3 = client33.get_collection(name=collection_name3)

In [5]:
blast_dna_questions = ["What is the word size parameter in BLAST?",
"How to get the results of BLASTP in XML format?",
"How to perform a BLAST on a specific taxonomic group?",
"What parameters do I use to perform BLAST with epitopes smaller than 10 amino acids?",
"Which kind of databases can be searched with BLASTX?"]

In [15]:
# an example prompt
prompt = blast_dna_questions[0]

# generate an embedding for the prompt and retrieve the most relevant doc
response = ollama.embeddings(
  prompt=prompt,
  model="mxbai-embed-large"
)
results = collection.query(
  query_embeddings=[response["embedding"]],
  n_results=3
)
data = results['documents'][0][0]
results

{'ids': [['page_25.txt', 'page_82.txt', 'page_54.txt']],
 'distances': [[161.27996826171875, 189.7296905517578, 195.74293518066406]],
 'metadatas': [[None, None, None]],
 'embeddings': None,
 'documents': [['blast feature tom madden1 christiam camacho2 created june 23 2008 updated march 14 2021 task blastn blastp application task option option set parameter eg wordsize gap value typical value specific type search example megablast task optimized intraspecies comparison us large wordsize whereas blastn better suited interspecies comparison shorter wordsize task resemble program selection section blast web page preclude user setting option override specified task see appendix option commandline application documentation parameter value different task following task currently available program task name description blastp blastp traditional blastp compare protein query protein database blastpshort blastp optimized query shorter 30 residue blastn blastn traditional blastn requiring exact m

In [12]:
model_name="dmis-lab/biobert-base-cased-v1.2"
# Initialize BioBERT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
em = ge.get_embeddings(prompt[4], model, tokenizer)
results = collection3.query(
  query_embeddings=em.tolist(),
  n_results=3
)
results

{'ids': [['page_68.txt', 'page_48.txt', 'page_38.txt']],
 'distances': [[185.29751586914062, 186.61318969726562, 186.95620727539062]],
 'metadatas': [[None, None, None]],
 'embeddings': None,
 'documents': [['60 blast command line application user manual',
   '40 blast command line application user manual',
   '30 blast command line application user manual']],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents', 'distances']}